# Surface Mass Balance: XGB 

Import relevant Libaries 

In [1]:
%pip install pyarrow
%pip install plotly
%pip install feature_engine
%pip update pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "update"
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Base libraries
import pandas as pd
import numpy as np
import os
import math as math
import datetime
from scipy import stats

# Visualizations
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns

# Set WD
import os
from pyrsistent import v
os.getcwd()

'E:\\Nils\\Master_Thesis\\scripts'

#### Load KAN_L 

In [15]:
#df = pd.read_csv(r'..\data\promice\preprocessed\daily\KAN_L.csv', index_col=0)
df = pd.read_csv(r'../data/promice/preprocessed/daily/SCO_L.csv', index_col=0)

#### Load SCO_L 

In [16]:
#df = pd.read_csv(r'..\data\promice\preprocessed\daily\KAN_L.csv', index_col=0)
df_1 = pd.read_csv(r'../data/promice/preprocessed/daily/KAN_L.csv', index_col=0)

df = df.append(df_1)

##### Pre-processing (0.1: Change months to cycle)

In [17]:
# This bit to change the datetime variables into a form that a ML models can understand
  # https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
  # https://feature-engine.trainindata.com/en/1.3.x/user_guide/creation/CyclicalFeatures.html
    
    
# This bit to split Datetime column into year, month, day, hour columns
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%d')

df['Datetime'] = pd.to_datetime(df['Datetime'])

#Create new columns
#df['year'] = df['Datetime'].dt.year 
df['month'] = df['Datetime'].dt.month
#df['day'] = df['Datetime'].dt.day
#df['hour'] = df['Datetime'].dt.hour

#Drop the datetime column
df.drop(columns=['Datetime'], inplace=True)

In [18]:
# This bit to change the datetime variables into a form that a ML models can understand (continued)
  # https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
  # https://feature-engine.trainindata.com/en/1.3.x/user_guide/creation/CyclicalFeatures.html 

# Use CyclicalFeatures Package to convert year, month, day & hour
from feature_engine.creation import CyclicalFeatures

cyclical = CyclicalFeatures(variables=None, drop_original=True)

# with year
#cyclical = cyclical.fit_transform(df[['month', 'year', 'Wind from direction (upper boom)']]) 

# Remove only values from Wind direction because CyclicalFeatures does not work with NAN
df = df[df['Wind from direction (upper boom)'].notna()] 

# without year
#cyclical = cyclical.fit_transform(df[['month', 'Wind from direction (upper boom)']]) 
cyclical = cyclical.fit_transform(df[['Wind from direction (upper boom)']]) 
df = pd.merge(df, cyclical, left_index=True, right_index=True)

df.drop(columns=['month', 'Wind from direction (upper boom)'], inplace=True)

##### Pre-processing (1.0: Filter relevant periods)

In [19]:
# Fill NA values in Melting season as blank
df['Melting Season'] = df['Melting Season'].fillna('no_ablation')

In [20]:
#Only Ablation Period Or Melting Season
df = df[df['Ablation'] == True]
#df =  df[df['Melting Season'].isin(['pre', 'beginning', 'middle', 'end'])]

#Remove all NA values if training should be done only on ablation
df = df.dropna()

df = df.reset_index(drop=True)

In [21]:
# This bit retrieve the indexes for all melting season categories and remove column from df
for category in ['no_ablation', 'pre', 'beginning', 'middle', 'end']:
    exec(f"indices_{category} = df[df['Melting Season'] == '{category}'].index")

# This bit retrieve the indexes for Ablation periods
for category in [True]:
    exec(f"indices_ablation = df[df['Ablation'] == {category}].index")

indices_all = df[df['Ablation'].isin([True, False])].index

##### Pre-processing (2.0: Formatting and transformation)

In [22]:
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
df['Melting Season'] = le.fit_transform(df['Melting Season'])
label_dict = dict(zip(le.classes_, le.transform(le.classes_)))

class_order = ['no_ablation', 'pre', 'beginning', 'middle', 'end']

# Sort the label_dict based on the desired order of class names
label_dict_sorted = {class_name: label_dict[class_name] for class_name in class_order if class_name in label_dict}

In [23]:
# This bit to include only relevant features
exclude_list = [                                                    'index', # excluded because of unimportant information
                                                                     'stid', # excluded because of unimportant information
                                                               'Unnamed: 0', # Old Index
                                                                 'Ablation',
                                                          # 'Melting Season', # We drop melting season later 
                                              'year', 'month','day', 'hour', # Drop because of sin & cos values
                                                                 'subgroup',
                'Surface height from combined measurements',
                'Surface height from combined measurements DELTA'
               ]

df = df[[column for column in df.columns if column not in exclude_list]]


In [24]:
# This bit to convert all numerical values into the same scale 
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]

# with year
#num_cols = df.select_dtypes(include=['int64','float64']).drop(['Surface height from combined measurements', 'Surface height from combined measurements DELTA','month_sin','month_cos','year_sin','year_cos','Wind from direction (upper boom)_sin', 'Wind from direction (upper boom)_cos', 'Melting Season'], axis=1).columns #select numerical columns except y and cyclical feature

# without year
#num_cols = df.select_dtypes(include=['int64','float64']).drop(['month_sin','month_cos','Wind from direction (upper boom)_sin', 'Wind from direction (upper boom)_cos', 'Melting Season'], axis=1).columns #select numerical columns except y and cyclical feature
num_cols = df.select_dtypes(include=['int64','float64']).drop(['Wind from direction (upper boom)_sin', 'Wind from direction (upper boom)_cos'], axis=1).columns #select numerical columns except y and cyclical feature

scaler = MinMaxScaler()

#fit and transform numerical columns
df[num_cols] = scaler.fit_transform(df[num_cols])

In [25]:
# This bit to split into Train & Test
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# split the data into training and testing sets while maintaining the subgroup distribution
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df, df['Melting Season']):
    train = df.loc[train_index]
    test = df.loc[test_index]

# separate the features and target variables for the train and test sets
X_train = train.drop(['Melting Season'], axis=1)
y_train = train['Melting Season']
X_test = test.drop(['Melting Season'], axis=1)
y_test = test['Melting Season']



In [33]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

labels = [k for k, v in sorted(label_dict.items(), key=lambda item: item[1])]

# Define a dictionary of models and their hyperparameters
models = {
    'Multinomial Regression': LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
    'K-Nearest Neighbors':  KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='linear', C=1.0),
    'RBF SVM':  SVC(kernel='rbf', C=1.0, gamma='auto'),
    'Gaussian Process': GaussianProcessClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Neural Net': MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000),
    'AdaBoost': AdaBoostClassifier(),
    'Naive Bayes': GaussianNB(),
    'QDA': QuadraticDiscriminantAnalysis()
}

# Loop through each model
for name, model in models.items():
    print(f"Evaluating {name}")
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Evaluate the model's performance on the test data
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy score on test set: {accuracy}")
    print(f"F1 score on test set: {f1}")
    print("Confusion matrix on test set: ")
    print(pd.DataFrame(cm, index=labels, columns=labels))
    print("Accuracy per class:")
    for i in range(len(labels)):
        acc = accuracy_score(y_test[y_test == i], y_pred[y_test == i])
        print(f"Class {i}: {acc}")
    print("--------------------")


Evaluating Multinomial Regression
Accuracy score on test set: 0.8361650485436893
F1 score on test set: 0.8205935579509974
Confusion matrix on test set: 
           beginning  end  middle
beginning         52    0      77
end                0   82      37
middle            14    7     555
Accuracy per class:
Class 0: 0.40310077519379844
Class 1: 0.6890756302521008
Class 2: 0.9635416666666666
--------------------
Evaluating K-Nearest Neighbors
Accuracy score on test set: 0.8118932038834952
F1 score on test set: 0.8029524291558784
Confusion matrix on test set: 
           beginning  end  middle
beginning         63    0      66
end                0   75      44
middle            34   11     531
Accuracy per class:
Class 0: 0.4883720930232558
Class 1: 0.6302521008403361
Class 2: 0.921875
--------------------
Evaluating SVM
Accuracy score on test set: 0.8361650485436893
F1 score on test set: 0.8172117501271972
Confusion matrix on test set: 
           beginning  end  middle
beginning       

In [35]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(100,), (100, 50), (200, 100, 50)],
    'activation': ['relu', 'tanh', 'logistic'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'early_stopping': [True, False],
   # 'dropout': [0.0, 0.1, 0.2],
}

# Define the neural network model
model = MLPClassifier(max_iter=1000)

# Define the grid search object
grid_search = GridSearchCV(model, param_grid = param_grid, cv=5, n_jobs=-1, verbose=1)

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy score on the test set
print("Best hyperparameters: ", grid_search.best_params_)
y_pred = grid_search.predict(X_test)
print("Accuracy score on test set: ", accuracy_score(y_test, y_pred))
print("F1 score on test set: ", f1_score(y_test, y_pred, average='weighted'))

labels = [k for k, v in sorted(label_dict.items(), key=lambda item: item[1])]
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion matrix on test set: ")
print(cm_df)
print("Accuracy per class:")
for i in range(len(labels)):
    acc = accuracy_score(y_test[y_test == i], y_pred[y_test == i])
    print(f"Class {i}: {acc}")
print("--------------------")

# Save the best model parameters as 'model' variable
model = grid_search.best_estimator_


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best hyperparameters:  {'activation': 'relu', 'alpha': 0.001, 'early_stopping': False, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.001}
Accuracy score on test set:  0.8580097087378641
F1 score on test set:  0.8557268635729134
Confusion matrix on test set: 
           beginning  end  middle
beginning         86    0      43
end                0   92      27
middle            27   20     529


In [40]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score, classification_report

# Define the hyperparameter grid for tuning
param_grid = {
    'reg_param': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
    'priors': [None, [0.3, 0.3, 0.4], [0.2, 0.2, 0.6]]
}

# Define the cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

# Define the QDA classifier
qda = QuadraticDiscriminantAnalysis()

# Define the grid search object
grid_search = GridSearchCV(qda, param_grid=param_grid, cv=cv, scoring=scoring, refit='accuracy', verbose=1, n_jobs=-1)

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding scores on the test set
print("Best hyperparameters: ", grid_search.best_params_)
print("Accuracy score on test set: ", grid_search.best_score_)
print("F1 score on test set: ", grid_search.cv_results_['mean_test_f1_score'][grid_search.best_index_])

# Evaluate the performance on the test set using the best estimator
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred, target_names=labels))


print("Accuracy per class:")
for i in range(len(labels)):
    acc = accuracy_score(y_test[y_test == i], y_pred[y_test == i])
    print(f"Class {i}: {acc}")
print("--------------------")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best hyperparameters:  {'priors': None, 'reg_param': 0.0, 'store_covariance': True, 'tol': 0.0001}
Accuracy score on test set:  0.8250974025974026
F1 score on test set:  0.8254205938453965
              precision    recall  f1-score   support

   beginning       0.63      0.71      0.67       129
         end       0.80      0.71      0.76       119
      middle       0.88      0.87      0.88       576

    accuracy                           0.83       824
   macro avg       0.77      0.77      0.77       824
weighted avg       0.83      0.83      0.83       824

Accuracy per class:
Class 0: 0.7131782945736435
Class 1: 0.7142857142857143
Class 2: 0.8732638888888888
--------------------


In [46]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Define a function that returns a compiled neural network model
def create_model(learning_rate=0.01, dropout_rate=0.0):
    # Create a sequential model
    model = Sequential()
    
    # Add a fully connected layer with 100 neurons and input shape of the number of features
    model.add(Dense(100, input_dim=X_train.shape[1], activation='relu'))
    
    # Add dropout layer
    model.add(Dropout(dropout_rate))
    
    # Add another fully connected layer with 50 neurons
    model.add(Dense(50, activation='relu'))
    
    # Add a final fully connected layer with number of neurons equal to the number of classes and softmax activation
    model.add(Dense(len(label_dict), activation='softmax'))
    
    # Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define a grid of hyperparameters to search over
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.0, 0.1, 0.2],
    'batch_size': [16, 32],
    'epochs': [50, 100]
}

# Create a grid search object with the model and hyperparameters
model = KerasClassifier(build_fn=create_model, verbose=0)
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)

# Fit the grid search object to the training data
grid_result = grid_search.fit(X_train, tf.keras.utils.to_categorical(y_train))

# Print the best hyperparameters and the corresponding accuracy score on the test set
print(f"Best hyperparameters: {grid_result.best_params_}")
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy score on test set: {accuracy}")
print(f"F1 score on test set: {f1}")
print("Confusion matrix on test set: ")
print(pd.DataFrame(cm, index=labels, columns=labels))


C:\Users\nifu18ab\AppData\Local\Temp\2/ipykernel_8956/3815655594.py:39: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best hyperparameters: {'batch_size': 32, 'dropout_rate': 0.2, 'epochs': 100, 'learning_rate': 0.001}
Accuracy score on test set: 0.8434466019417476
F1 score on test set: 0.8388609101209606
Confusion matrix on test set: 
           beginning  end  middle
beginning         80    0      49
end                1   81      37
middle            29   13     534


In [47]:
print("Accuracy per class:")
for i in range(len(labels)):
    acc = accuracy_score(y_test[y_test == i], y_pred[y_test == i])
    print(f"Class {i}: {acc}")
print("--------------------")

Accuracy per class:
Class 0: 0.6201550387596899
Class 1: 0.680672268907563
Class 2: 0.9270833333333334
--------------------


--------